In [ ]:
import os
import folium

def compute_overlap(bbox1, bbox2):
    length = max(min(bbox2[2] - bbox1[0], bbox1[2] - bbox2[0]), 0)
    width = max(min(bbox2[3] - bbox1[1], bbox1[3] - bbox2[1]), 0)
    return length * width


for city in os.listdir(os.path.join("dataset", "splits")):
    city_splits_path = os.path.join("dataset", "splits", city)
    aer_overlap_dict = {}
    gl_overlap_dict = {}
    with open(os.path.join(city_splits_path, "samples.csv"), "r") as f:
        lines = f.readlines()
        bbox_dict = {}

        for line in lines[1:]:
            aer_image_name = line.strip().split(',')[0]

            bbox = aer_image_name[:-4].split("_")[1:]
            bbox = [float(edge) for edge in bbox]
            bbox_dict[aer_image_name] = bbox

        for sample1 in lines[1:]:
            sample1_images = sample1.strip().split(',')
            sample1_aerial = sample1_images[0]
            sample1_gls = set(sample1_images[1:])

            aer_overlap = {}
            gl_overlap = {}

            sample1_bbox = bbox_dict[sample1_aerial]

            area = (sample1_bbox[2] - sample1_bbox[0]) * (sample1_bbox[3] - sample1_bbox[1])

            for sample2 in lines[1:]:
                sample2_images = sample2.strip().split(',')
                sample2_aerial = sample2_images[0]
                sample2_gls = set(sample2_images[1:])

                aer_perc_overlap = compute_overlap(sample1_bbox, bbox_dict[sample2_aerial]) / area
                gl_perc_overlap = len(sample1_gls & sample2_gls) / len(sample1_gls)

                aer_overlap[sample2_aerial] = aer_perc_overlap
                gl_overlap[sample2_aerial] = gl_perc_overlap

            aer_overlap_dict[sample1_aerial] = aer_overlap
            gl_overlap_dict[sample1_aerial] = gl_overlap

    for item in aer_overlap_dict:
        print(f"{item}: {sum(aer_overlap_dict[item].values())}")
    # with open("overlap.csv", "w") as f:
    #     f.write("")